In [1]:
import re
import pickle
import math
import os
import pandas as pd
import rouge
import codecs

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AdamW
from tqdm import tqdm

from src.model.logger import Logger
from src.model.data_full import RawFilesDataset
from src.model.loss import ParagraphLoss
from src.model.generate_utils import toks_to_str

from rake_nltk import Rake
from nltk.corpus import stopwords
import csv

# from src.model.generate_utils  import generate_paragraph
# from src.model.eval_utils import evaluate_doc_model
# from src.model.model import GPT2BaseModel


# generate

In [ ]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()


In [ ]:
str_rep = []
end_tok = encoder.convert_tokens_to_ids('_end_')

for token in sample_output[0]:
    print(token.item(), repr(decoder.decode([ encoder.convert_ids_to_tokens(token.item(), skip_special_tokens=True)])))
    if token.item() == end_tok : #or token.item() == 0:# or x.item() == end_idx:
        break        
    str_rep.append(encoder.convert_ids_to_tokens(token.item()))

str_rep = encoder.convert_tokens_to_string(str_rep)

# This makes sure rouge scorers doesn't complain about no sentences
if not str_rep:
    str_rep = "unk."
elif "." not in str_rep:
    str_rep += "."

print(encoder.decode(sample_output[0], skip_special_tokens=False, clean_up_tokenization_spaces=False))
print("-"*50)
print(str_rep)

# evaluate doc

In [ ]:
class Config:
    repeattheta = 1.5
    output_attentions = True

In [ ]:
args = Config()

In [ ]:
vocab = len(encoder)

In [ ]:
doc_model = GPT2BaseModel(args, vocab=vocab, n_ctx=config['n_ctx'], gen_len=401, lastidx=encoder.eos_token_id, includeprev=False, device='cpu')

In [ ]:
evaluate_doc_model(model=doc_model, val_loader=val_loader, text_encoder=encoder, device='cpu', beam=0, gen_len=401, k=0, p=90, save_file='out', max_len=512, gen_dir=None, tgt_dir=None, min_len=100)

In [ ]:
import json

In [ ]:
with open('text.txt', 'w', encoding='utf-8') as f:
    json.dump("Моя строка", f, ensure_ascii=False)

In [ ]:
df = pd.read_csv('generated/test.gens.tsv', sep='\t', header=None, names=['id', 'plot', 'context', 'part', 'text'])
df.head()

In [ ]:
df.text[0]

# Generate

## rake

* для токенизаторов в исходном коде не применяется язык
* англ токенизатор предложений лучше делит, к примеру русский не смог разделить 'Король дал за дочкой богатое приданое, наградил зятя большим чином и задал пир на весь мир.\nЖивут молодые месяц, и два, и три.'

In [ ]:
from rake_nltk import Metric, Rake

In [ ]:
story = '111 Волшебное кольцо.txt'
path = 'dataset/raw'
with open(os.path.join(path, story), 'r', encoding='utf-8') as f:
    text =  f.read()
    text = re.sub('\.\.\.', '.', text)
    text = re.sub('—', '-', text)

## metrics

In [ ]:
def rouge_scores(hyps, refs):       
    rouge_scorer = rouge.Rouge()
    averaged_scores = rouge_scorer.get_scores(hyps, refs, avg=True)
    return averaged_scores

## samples

In [11]:
text_encoder = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", add_prefix_space=True)
text_encoder.add_special_tokens({'bos_token': '<s>',                                     
                                    'eos_token': '</s>',
                                    'additional_special_tokens': ['[SEP]', '_kw_', '_endkw_']
                                })

3

In [12]:
with open('savedir/s_all_nodiscourse_kw/checkpoints/checkpoint.pt', 'rb') as f:
    model = torch.load(f, map_location=torch.device('cpu'))

with open('savedir/s_all_nodiscourse_kw/test_dataset', 'rb') as f:
    test = pickle.load(f)

In [13]:
test_dataset = RawFilesDataset(test, text_encoder, 2048, n_ctx=70)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [14]:
model.eval()
batch = next(iter(test_loader))

In [15]:
input_ids, mask = batch['sample'], batch['mask']

In [18]:
target = input_ids[:, 70:]
mask_tgt = mask[:, 70:]
new = target[mask_tgt==1]

In [15]:
texts = text_encoder.batch_decode(batch['sample'], clean_up_tokenization_spaces=False, skip_special_tokens=False)

In [ ]:
text_encoder.additional_special_tokens

In [ ]:
print(list(batch['mask'][0,0:70].numpy()))

In [36]:
path = 'savedir/s_all_nodiscourse_kw/'

In [37]:
df = pd.read_csv(os.path.join(path, 'generated_stories.csv'), sep='|')

In [41]:
print(df.refs[1])

Помер старик со старухою, оставался у них сын сирота. Взял его к себе дядя и заставил овец пасти. Ни много, ни мало прошло времени, призывает дядя племянника, хочет попытать у него ума-разума и говорит ему: - Вот тебе сотни баранов, гони их на ярмонку да продай с барышом, чтобы и бараны были целы, и деньги сполна выручены. Что тут делать! Заплакал бедняга и погнал баранов в чистое поле: выгнал, сел на дороге и задумался о своем горе. Идет мимо девица: - О чем слезы льешь, добрый молодец? - Как же мне не плакать? Нет у меня ни отца, ни матери; один дядя, и тот обижает! - Какую ж обиду он тебе делает? - Да вот послал на ярмонку, велел баранами торговать, да так, чтобы и бараны были целы, и деньги сполна выручены. - Ну, это хитрость не великая! Найми-ка ты баб да остриги баранов, а шерсть отнеси на ярмонку и продай; вот у тебя и деньги, и бараны в целости! Парень так и сделал; продал шерсть, пригнал стадо домой и отдает дяде вырученные деньги. - Хорошо, - говорит дядя племяннику, - только

In [31]:
text = '<s> царь тотчас снял кольцо _kw_ кот васька говорит _kw_ кот взял кольцо _kw_ днем королевна носит кольцо _kw_'

tokens = text_encoder.encode(text)
hyps = text_encoder.decode(tokens, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(hyps)

царь тотчас снял кольцокот васька говориткот взял кольцоднем королевна носит кольцо
